# Writing Deschutes NF multi-objective CPLEX model file

In [28]:
import pandas as pd
import numpy as np

In [29]:
# Define object that is a body of all required data for the model file
class DeschutesModelData:
    def __init__(self, areas, fire, sed, clusters, nsoCands1, nsoCands2,
                 maxPerPeriodTrt, interPeriodFluctuation, nsoHabSizeDiscount):
        self.areas = areas
        self.stands = areas["stand"]
        self.fire = fire
        self.climProjections = fire["climateScenario"].unique().tolist()
        self.sed = sed
        self.clusters = clusters
        self.nsoCands = {}
        self.nsoCands["1"] = nsoCands1
        self.nsoCands["2"] = nsoCands2
        self.maxPerPeriodTrt = maxPerPeriodTrt
        self.interPeriodFluctuation = interPeriodFluctuation
        self.nsoHabSizeDiscount = nsoHabSizeDiscount

In [30]:
# Instantiate a DeschutesModelData object with our data files
dmd = DeschutesModelData(areas = pd.read_csv("data/standAreas.csv"),
                         fire = pd.read_csv("data/fire.csv"),
                         sed = pd.read_csv("data/sed_wClim.csv"),
                         clusters = pd.read_csv("data/clusters.csv"),
                         nsoCands1 = pd.read_csv("data/standNSOCandidacy_wTrtTimes_raw.csv"), # make new files with subscript
                         nsoCands2 = pd.read_csv("data/standNSOCandidacy_wTrtTimes_raw.csv"), # for times 1, 2
                         maxPerPeriodTrt = 2428.114, #6000 acres to ha
                         interPeriodFluctuation = 0.2,
                         nsoHabSizeDiscount = 0.5)

In [31]:
# Method writes the constraint defining the fire objective
def writeFireObjDefinition(model, fireTable):
    model.write("FireObjective: ")
    for index, row in fireTable.iterrows(): # for each stand
        s = str(int(row.stand))
        for idx, val in enumerate(row["none":]): # and for each treatment scenario,
            # write obj fn terms for associated reduction in fire hazard
            if val < 0:
                model.write(" " + str(val) + " x_" + s + "_" + str(idx))
            else:
                model.write(" + " + str(val) + " x_" + s + "_" + str(idx))
        model.write("\n") # move on to next stand
    model.write(" + FireHazardIncrease = 0\n") # terminate constraint

In [32]:
# Method writes the constraint defining the owl objective
# See following cell for new definition
def writeOwlObjDefinition(model, owlTable, areas, discount):
    owl = owlTable.merge(areas, how="left", on="stand")
    owl = owl[["stand","area (ha)","none","first","second","both"]]
    model.write("OwlObjective: ")
    for index, row in owl.iterrows():
        s = str(int(row["stand"]))
        a = row["area (ha)"]
        if sum(row["none":]) > 0: # If the stand qualifies as NSO habitat in at least one treatment scenario...
            # Then the site enters the objective function. We begin by writing its trigger var's for p
            # (the following write statement could be shortened, but leaving it as two terms makes the logic more obvious)
            model.write(" + " + str(a) + " p_" + s + " - " + str(discount*a) + " p_" + s)
            for idx, val in enumerate(row["none":]): # Next we sum up over the treatment scenarios...
                if val > 0: # in which the stand qualifies as NSO habitat (val will be 1)
                    model.write(" + " + str(discount*a) + " x_" + s + "_" + str(idx))
            model.write("\n") # move on to next stand
    model.write(" - OwlHabitat = 0\n") # terminate constraint               

In [33]:
### UNDER DEVELOPMENT - REPLACES ABOVE CELL
### TRANSITIONING TO PER-PERIOD MESAUREMENT OF OWL HAB (AND MAX'ING THIS VALUE)
# Method writes the constraint defining the owl objective
# Under new definition of owl habitat, this will have to be done twice
def writeOwlHabDef1(model, owlTablep1, areas, discount):
    owl = owlTablep1.merge(areas, how="left", on="stand")
    owl = owl[["stand","area (ha)","none","first","second","both"]]
    model.write("OwlHabitat_2035: ")
    for index, row in owl.iterrows():
        s = str(int(row["stand"]))
        a = row["area (ha)"]
        if sum(row["none":]) > 0: # If the stand qualifies as NSO habitat in at least one treatment scenario...
            # Then the site enters the objective function. We begin by writing its trigger var's for p
            # (the following write statement could be shortened, but leaving it as two terms makes the logic more obvious)
            model.write(" + " + str(a) + " p_" + s + "_1 " + " - " + str(discount*a) + " p_" + s + "_1")
            for idx, val in enumerate(row["none":]): # Next we sum up over the treatment scenarios...
                if val > 0: # in which the stand qualifies as NSO habitat (val will be 1)
                    model.write(" + " + str(discount*a) + " x_" + s + "_" + str(idx))
            model.write("\n") # move on to next stand
    model.write(" - OwlHabitat_1 = 0\n") # terminate constraint
    model.write("MinOwlHabitatLEOwlHab1: MinOwlHabitat - OwlHabitat_1 <= 0\n")

def writeOwlHabDef2(model, owlTablep2, areas, discount):
    owl = owlTablep2.merge(areas, how="left", on="stand")
    owl = owl[["stand","area (ha)","none","first","second","both"]]
    model.write("OwlHabitat_2055: ")
    for index, row in owl.iterrows():
        s = str(int(row["stand"]))
        a = row["area (ha)"]
        if sum(row["none":]) > 0: # If the stand qualifies as NSO habitat in at least one treatment scenario...
            # Then the site enters the objective function. We begin by writing its trigger var's for p
            # (the following write statement could be shortened, but leaving it as two terms makes the logic more obvious)
            model.write(" + " + str(a) + " p_" + s + "_2 " + " - " + str(discount*a) + " p_" + s + "_2")
            for idx, val in enumerate(row["none":]): # Next we sum up over the treatment scenarios...
                if val > 0: # in which the stand qualifies as NSO habitat (val will be 1)
                    model.write(" + " + str(discount*a) + " x_" + s + "_" + str(idx))
            model.write("\n") # move on to next stand
    model.write(" - OwlHabitat_2 = 0\n") # terminate constraint
    model.write("MinOwlHabitatLEOwlHab2: MinOwlHabitat - OwlHabitat_2 <= 0\n")
    

In [34]:
# Method writes the constraints defining the sediment objective
def writeSedimentObjDefinition(model, sedTable):
    # Begin with defining the increased sediment delivery in the first period, S1
    model.write("DefineS1: ")
    for index, row in sedTable.iterrows(): # for each stand
        s = str(int(row.stand))
        c1 = str(row.trtIn1) # contribution to increased sediment delivery in first period (above baseline)
        model.write(" + " + c1 + " x_" + s + "_1" + " + " + c1 + " x_" + s + "_3\n")
    model.write(" - S1 = 0\n")
    # Now define the increased sediment delivery in the second period, S2
    model.write("DefineS2: ")
    for index, row in sedTable.iterrows(): # for each stand
        s = str(int(row.stand))
        c2 = str(row.trtIn2) # contribution to increased sediment delivery in second period (above baseline)
        model.write(" + " + c2 + " x_" + s + "_2" + " + " + c2 + " x_" + s + "_3\n")
    model.write(" - S2 = 0\n")
    # Finally, require the objective MaxSediment to be the max of these two
    model.write("MaxSedimentGES1: MaxSediment - S1 >= 0\n")
    model.write("MaxSedimentGES2: MaxSediment - S2 >= 0\n")

In [35]:
# Method writes the constraints defining the cluster variable triggers, q_c
def writeClusterVarTriggers(model, dmd, clim):
    for t in ["1","2"]:
        for index, row in dmd.clusters.iterrows(): # for each cluster...
            c = str(int(row.CL))
            stands = row["S1":][row["S1":] > 0] # get list of sites in the cluster (0 indicating no site in the cluster file)
            k = str(len(stands)) # store cardinality of the cluster
            model.write("QVarTrigger_" + c + "_" + t + ": -" + k + " q_" + c + "_" + t) # write intro to constraint and single term for the trigger var
            for stand in stands: # Then for each stand in the cluster...
                s = str(int(stand))
                # Get the list of treatment prescriptions for which the stand is suitable NSO habitat:
                owlQual = dmd.nsoCands[t].loc[dmd.nsoCands[t].stand == stand].loc[dmd.nsoCands[t].climateScenario == clim].loc[:,"none":].iloc[0]
                for idx, val in enumerate(owlQual):
                    if val > 0:
                        model.write(" + x_" + s + "_" + str(idx)) # And for those stand-treatment prescriptions, write a term in the constraint
                model.write("\n") # Done with one stand in the cluster. Jump to new line to begin the next
            model.write(" >= 0\n") # Complete cluster constraint

In [36]:
# Method writes constraint for site-wise trigger variables p_i_t
def writeOwlClusterSiteTriggers(model, dmd, clim):
    for t in ["1","2"]:
        for idx, row in dmd.nsoCands[t].loc[dmd.nsoCands[t].climateScenario == "NONE"].iterrows(): # For each stand...
            s = str(int(row.stand))
            if sum(row["none":]) > 0:
                model.write("PVarTrigger_" + s + "_" + t + ": -p_" + s + "_" + t)
                clustersWThisStand = dmd.clusters.loc[(dmd.clusters.loc[:,"S1":] == int(s)).any(axis=1)]
                for idxc, rowc in clustersWThisStand: # get list of clusters that contain this stand
                    cl = str(int(rowc.CL))
                    model.write(" + q_" + cl + "_" + t +"\n")
                model.write(" >= 0\n") # Done adding clusters to the constraint, so finish it.

In [37]:
# Method writes constratint requiring exactly one treatment prescription per site
def writeOnePrescripPerSite(model, dmd):
    # We will iterate through all stands once. We need access to the treatment prescriptions, so we use a class attribute that contains them - we chose fire here
    for index, row in dmd.fire.loc[dmd.fire.climateScenario == "NONE"].iterrows():
        s = str(int(row.stand))
        model.write("OneTrtPrescription_" + s + ": ")
        for idx, val in enumerate(row["none":]):
            model.write(" + x_" + s + "_" + str(idx))
        model.write(" = 1\n")

In [38]:
# Method writes constraints defining and restricting the area treated in each time period
def writeMaxAreaTreatedThresh(model, dmd):
    # Defining the area treated in the first time period
    model.write("DefineTreatedArea_1: ")
    for index, row in dmd.areas.iterrows(): # for each stand...
        s = str(int(row["stand"]))
        a = str(row["area (ha)"])
        model.write(" + " + a + " x_" + s + "_1" + " + " + a + " x_" + s + "_3\n")
    model.write(" - A1 = 0\n")
    # Restricting first period area
    model.write("RestrictMaxArea_1: A1 <= " + str(dmd.maxPerPeriodTrt) + "\n")
    # Defining the treated area in the second time period
    model.write("DefineTreatedArea_2: ")
    for index, row in dmd.areas.iterrows(): # for each stand...
        s = str(int(row["stand"]))
        a = str(row["area (ha)"])
        model.write(" + " + a + " x_" + s + "_2" + " + " + a + " x_" + s + "_3\n")
    model.write(" - A2 = 0\n")
    model.write("RestrictMaxArea_2: A2 <= " + str(dmd.maxPerPeriodTrt) + "\n")

In [39]:
# Method writes constraint restricting the inter-period treatment area fluctuation
def writeTreatedAreaFluctThresh(model,dmd):
    model.write("LowerBoundOnFluctuation: " + str(1 - dmd.interPeriodFluctuation) + " A1 - A2 <= 0\n")
    model.write("UpperBoundOnFluctuation: " + str(1 + dmd.interPeriodFluctuation) + " A1 - A2 >= 0\n")

In [40]:
# CPLEX model files end with a statement declaring which variables are general (integer), binary, etc.
# Write such constraints for our model
def writeBinaryVarStmt(model, dmd, clim):
    # Write for the X vars
    for index, row in dmd.fire.loc[dmd.fire.climateScenario == clim].iterrows():
        s = str(int(row.stand))
        for idx,val in enumerate(row["none":]):
            model.write("x_" + s + "_" + str(idx) + "\n")
    # Write for the P and Q vars
    for t in ["1","2"]:
        for index, row in dmd.nsoCands[t].loc[dmd.nsoCands[t].climateScenario == clim].iterrows(): # For each stand...
            s = str(int(row["stand"]))
            if sum(row["none":]) > 0:
                model.write("p_" + s + "_" + t + "\n")
        for index, row in dmd.clusters.iterrows():
            c = str(int(row.CL))
            model.write("q_" + c + "_" + t + "\n")

In [41]:
# Larger umbrella methods for the writing of the CPLEX model file
def writeObjective(model):
    model.write("MAXIMIZE\n")
    model.write("OBJECTIVE: -FireHazardIncrease + MinOwlHabitat - MaxSediment\n\n")

def writeConstraints(model, clim, dmd):
    model.write("Subject To:\n")
    writeFireObjDefinition(model, dmd.fire.loc[dmd.fire.climateScenario == clim])
    #writeOwlObjDefinition(model, dmd.nsoCands.loc[dmd.nsoCands.climateScenario == clim], dmd.areas, dmd.nsoHabSizeDiscount)
    writeOwlHabDef1(model, dmd.nsoCands["1"].loc[dmd.nsoCands["1"].climateScenario == clim], dmd.areas, dmd.nsoHabSizeDiscount)
    writeOwlHabDef2(model, dmd.nsoCands["2"].loc[dmd.nsoCands["2"].climateScenario == clim], dmd.areas, dmd.nsoHabSizeDiscount)
    writeSedimentObjDefinition(model, dmd.sed.loc[dmd.sed.climate == clim])
    writeClusterVarTriggers(model, dmd, clim) # the per-cluster constraints
    writeOwlClusterSiteTriggers(model, dmd, clim) # the per-site constraints on the value of p_i
    writeOnePrescripPerSite(model, dmd)
    writeMaxAreaTreatedThresh(model, dmd) # per time period, sum of area treated < maxPerPeriodTrt
    writeTreatedAreaFluctThresh(model, dmd) # difference in period to period treatment areas
    model.write("\n")
    
def writeVariableStatement(model, dmd, clim):
    model.write("BINARY\n")
    writeBinaryVarStmt(model, dmd, clim)
    model.write("\n")

In [ ]:
# Write set of CPLEX model files
for clim in dmd.climProjections:
    with open("modelFiles/testModel_NewOWlObj_" + clim + ".txt", "w") as model:
        model.write("\\ Deschutes NF model file for climate scenario " + clim + "\n")
        writeObjective(model)
        writeConstraints(model, clim, dmd)
        writeVariableStatement(model, dmd, clim)
        model.write("\nEND\n")
    model.closed

In [45]:
# Experimentation cell